In [12]:
from kafka import KafkaProducer
from cassandra.cluster import Cluster
from cassandra.policies import DCAwareRoundRobinPolicy
from cassandra.cqlengine import connection
from cassandra.cqlengine import management
from cassandra.cqlengine.columns import Text, Map, List, DateTime
from cassandra.cqlengine.models import Model

In [3]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

In [4]:
producer.send('clicks', b'click')

In [5]:
producer.send('clicks', b'click').get()

RecordMetadata(topic='clicks', partition=0, topic_partition=TopicPartition(topic='clicks', partition=0), offset=7, timestamp=1558859867530, checksum=None, serialized_key_size=-1, serialized_value_size=5, serialized_header_size=-1)

In [6]:
cluster = Cluster(['172.17.0.3'])
session = cluster.connect()

In [7]:
connection.register_connection('cluster', session=session, default=True)
management.create_keyspace_simple(name='presidents', replication_factor=3)

/home/avi/anaconda3/lib/python3.7/site-packages/cassandra/cqlengine/management.py:545: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [11]:
DateTime.truncate_microseconds = False

class Click(Model):
    __keyspace__ = 'presidents'
    game_id = Text(primary_key=True)
    username = Text(primary_key=True)
    action_timestamps_dict = Map(key_type=Text(), value_type=List(value_type=DateTime()))

management.sync_table(Click)

/home/avi/anaconda3/lib/python3.7/site-packages/cassandra/cqlengine/management.py:545: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [13]:
for click in Click.objects().all():
    print(click)